In [36]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as stat
import statsmodels.tsa.stattools as ts
import csv
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()
import time

In [40]:
hist=cg.get_coin_market_chart_range_by_id(id='bitcoin',vs_currency='usd',from_timestamp='1295993242',to_timestamp=str(tf))
df=pd.DataFrame(hist['prices'])
df = df.rename(columns={0: 'time', 1: 'price'})
df_t=df.copy(deep=True)
df_t['time'] = pd.to_datetime(df_t['time'], unit='ms')
df_t.set_index('time', inplace=True)

In [133]:
t0='1295993242'

def get_price(tick,ti):
    tf=time.time()
    tf=int(tf)
    hist=cg.get_coin_market_chart_range_by_id(id=tick,vs_currency='usd',from_timestamp=ti,to_timestamp=str(tf))
    df=pd.DataFrame(hist['prices'])
    df = df.rename(columns={0: 'time', 1: 'close'})
    df_t=df.copy(deep=True)
    df_t['time'] = pd.to_datetime(df_t['time'], unit='ms')
    df_t.set_index('time', inplace=True)
    return df,df_t

def coint_test(tick1,tick2):
    A=get_price(tick1,t0)[0]
    B=get_price(tick2,t0)[0]
    ti_A=A['time'][0]
    ti_B=B['time'][0]
    if ti_A<ti_B:
        A=get_price(tick1,str(int(ti_B/1000)))
        B=get_price(tick2,str(int(ti_B/1000)))
    else:
        A=get_price(tick1,str(int(ti_A/1000)))
        B=get_price(tick2,str(int(ti_A/1000)))
    A_t=A[1]
    B_t=B[1]
    A=A[0]
    B=B[0]
    X=np.stack((np.log10(A['close']), np.log10(B['close'])), axis=0)
    cov=np.cov(X)
    #print(cov)
    covAB=cov[0,1]
    varA=cov[0,0]
    varB=cov[1,1]
    cor=covAB/(np.sqrt(varA*varB))
    lAB=covAB/varB #res=log(A)-lAB*log(B)
    lBA=covAB/varA #res=log(B)-lBA*log(A)
    spreadAB=np.log10(A['close'])-lAB*np.log10(B['close'])
    muAB=np.mean(spreadAB)
    spreadBA=np.log10(B['close'])-lBA*np.log10(A['close'])
    muBA=np.mean(spreadBA)
    return cor,lAB,lBA,muAB,muBA,spreadAB,spreadBA

In [116]:
get_price('bitcoin','1438905600')

(               time         close
 0     1438905600000    278.508814
 1     1438992000000    259.800728
 2     1439078400000    264.338483
 3     1439164800000    263.578200
 4     1439251200000    269.866800
 ...             ...           ...
 2359  1642723200000  40707.682414
 2360  1642809600000  36508.799790
 2361  1642896000000  35180.435463
 2362  1642982400000  36306.409440
 2363  1643068800000  36774.007142
 
 [2364 rows x 2 columns],
                    close
 time                    
 2015-08-07    278.508814
 2015-08-08    259.800728
 2015-08-09    264.338483
 2015-08-10    263.578200
 2015-08-11    269.866800
 ...                  ...
 2022-01-21  40707.682414
 2022-01-22  36508.799790
 2022-01-23  35180.435463
 2022-01-24  36306.409440
 2022-01-25  36774.007142
 
 [2364 rows x 1 columns])